In [96]:
# import library you use
import pandas as pd

# Training phase

### Load Data

In [97]:
# load train.csv and train.csv
train = pd.read_csv("train.csv")
train_label = pd.read_csv("train_label.csv")
train

,mac_hash,sniffer_loc,created_time
0,10f6ad2e3fa3b3da8ceb3fbd95658681,2,2018-12-08 11:47:48
1,10f6ad2e3fa3b3da8ceb3fbd95658681,10,2018-12-08 12:37:34
2,10f6ad2e3fa3b3da8ceb3fbd95658681,9,2018-12-08 12:37:35
3,10f6ad2e3fa3b3da8ceb3fbd95658681,6,2018-12-08 12:44:48
4,10f6ad2e3fa3b3da8ceb3fbd95658681,4,2018-12-08 12:44:54
...,...,...,...
33222,13a3e10f950d8050ab5437cf30a3c2c6,4,2018-12-06 20:13:40
33223,13a3e10f950d8050ab5437cf30a3c2c6,9,2018-12-06 20:14:42
33224,13a3e10f950d8050ab5437cf30a3c2c6,3,2018-12-06 20:16:42
33225,13a3e10f950d8050ab5437cf30a3c2c6,1,2018-12-06 20:18:41


### Data Preprocessing

In [98]:
# drop created_time column
train = train.drop(['created_time'], axis=1)
train

,mac_hash,sniffer_loc
0,10f6ad2e3fa3b3da8ceb3fbd95658681,2
1,10f6ad2e3fa3b3da8ceb3fbd95658681,10
2,10f6ad2e3fa3b3da8ceb3fbd95658681,9
3,10f6ad2e3fa3b3da8ceb3fbd95658681,6
4,10f6ad2e3fa3b3da8ceb3fbd95658681,4
...,...,...
33222,13a3e10f950d8050ab5437cf30a3c2c6,4
33223,13a3e10f950d8050ab5437cf30a3c2c6,9
33224,13a3e10f950d8050ab5437cf30a3c2c6,3
33225,13a3e10f950d8050ab5437cf30a3c2c6,1


In [99]:
# Count whether each mac_hash has passed through these 14 locations, and generate a training dataframe
df = pd.DataFrame(train)
one_hot_loc = pd.get_dummies(df.loc[:,'sniffer_loc'])
df = df.drop(['sniffer_loc'], axis=1)
df = df.join(one_hot_loc)
mac = df.groupby('mac_hash', as_index=False).sum()
mac.head()

,mac_hash,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0003ae8541d0e925fcee242287e2ad27,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,00078611037990f7f36b722f22595fe7,0,1,1,1,1,1,0,1,0,1,1,0,0,0
2,000ce31739b333d931813de403156844,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,0027240b2aeb1f7f3c053c073cafb452,0,1,0,1,1,1,1,1,1,1,1,1,1,1
4,0028a89964416dc9a8a66cc7fe25ef4e,0,1,1,1,0,1,0,0,1,1,1,1,1,1


In [100]:
# Add label column
df1 = pd.DataFrame(train_label)
data_train = pd.merge(mac, df1)
data_train.head()

,mac_hash,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
0,0003ae8541d0e925fcee242287e2ad27,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,00078611037990f7f36b722f22595fe7,0,1,1,1,1,1,0,1,0,1,1,0,0,0,1
2,000ce31739b333d931813de403156844,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3
3,0027240b2aeb1f7f3c053c073cafb452,0,1,0,1,1,1,1,1,1,1,1,1,1,1,4
4,0028a89964416dc9a8a66cc7fe25ef4e,0,1,1,1,0,1,0,0,1,1,1,1,1,1,4


### Training

In [101]:
# X: feature from training dataframe
X = data_train[data_train.columns[1:15]]

In [102]:
# Y: label from training dataframe
# you need to change your label from categorical encoding to one-hot encoding
Y = data_train[data_train.columns[15]]
Y = pd.get_dummies(Y)
Y

,0,1,2,3,4
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
5563,0,0,0,0,1
5564,1,0,0,0,0
5565,0,0,0,0,1
5566,1,0,0,0,0


In [103]:
# split your dataset as training dataset and validation dataset
# using train_test_split function,
### training dataset : validation dataset = 8 : 2
### random_state = 0
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, train_size=0.8)
print(X_train)
print(X_test)


      1  2  3  4  5  6  7  8  9  10  11  12  13  14
253   0  1  1  1  1  1  1  1  1   1   1   1   1   0
1114  0  1  0  1  1  1  1  1  1   1   1   1   1   1
4257  0  1  1  1  1  1  0  0  1   1   1   1   1   0
3979  1  1  1  0  0  1  0  0  0   0   0   0   0   0
1012  0  1  0  0  1  0  0  0  0   0   0   1   0   0
...  .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..
4931  0  0  0  0  0  0  0  0  0   0   0   1   0   1
3264  0  1  1  1  1  1  1  1  1   0   1   1   1   0
1653  1  1  1  1  0  1  1  1  1   1   1   1   1   1
2607  0  0  0  1  0  1  0  0  0   0   0   0   0   0
2732  0  1  1  1  1  1  0  1  1   1   1   1   1   1

[4454 rows x 14 columns]
      1  2  3  4  5  6  7  8  9  10  11  12  13  14
3987  0  1  1  1  1  1  1  1  1   1   1   1   1   1
690   1  1  1  1  1  1  0  1  0   1   1   0   0   0
944   1  1  1  1  1  1  0  1  0   0   0   1   0   0
2565  0  1  1  0  0  0  0  0  0   0   0   1   0   1
4029  0  0  0  0  0  1  0  0  0   0   0   0   0   0
...  .. .. .. .. .. .. .. .. ..  ..  .

In [104]:
# build DNN model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

model = Sequential()
# 加入 hidden layer-1 of 512 neurons 並指定 input_dim 為 784
model.add(Dense(40, input_dim=14))
# 使用 'relu' 當作 activation function
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(70))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(21))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 40)                600       
                                                                 
 activation_20 (Activation)  (None, 40)                0         
                                                                 
 dropout_15 (Dropout)        (None, 40)                0         
                                                                 
 dense_21 (Dense)            (None, 70)                2870      
                                                                 
 activation_21 (Activation)  (None, 70)                0         
                                                                 
 dropout_16 (Dropout)        (None, 70)                0         
                                                                 
 dense_22 (Dense)            (None, 21)               

In [105]:
# start train model
# 定義訓練方式  
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [106]:
# validate your model using validation dataset
from sklearn import preprocessing
# 開始訓練  
train_history = model.fit(x=X_train, y=Y_train, validation_split=0.1, epochs=10, batch_size=40)  
# After prediction, you need to change your result and label from one-hot encoding to categorical encoding
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaledFeatures = minmax_scale.fit_transform(X_train)

score = model.evaluate(x=scaledFeatures, y=Y_train)
print ('\nTrain Loss:', score[0])
print ('\nTrain Acc:', score[1])

Epoch 1/10
101/101 [==============================] - 2s 5ms/step - loss: 1.3422 - accuracy: 0.4933 - val_loss: 0.8606 - val_accuracy: 0.7220
Epoch 2/10
101/101 [==============================] - 0s 3ms/step - loss: 0.7743 - accuracy: 0.7121 - val_loss: 0.4231 - val_accuracy: 0.8700
Epoch 3/10
101/101 [==============================] - 0s 3ms/step - loss: 0.5512 - accuracy: 0.7964 - val_loss: 0.2781 - val_accuracy: 0.9193
Epoch 4/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4362 - accuracy: 0.8446 - val_loss: 0.2284 - val_accuracy: 0.9350
Epoch 5/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3758 - accuracy: 0.8678 - val_loss: 0.2035 - val_accuracy: 0.9305
Epoch 6/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3342 - accuracy: 0.8852 - val_loss: 0.1936 - val_accuracy: 0.9395
Epoch 7/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3199 - accuracy: 0.8935 - val_loss: 0.1836 - val_accuracy: 0.9439
Epoch 

In [107]:
# using sklearn classification_report function to show your validation result
from sklearn.metrics import classification_report
import numpy as np 

exp = Y_train.idxmax(axis=1)
pred = np.argmax(model.predict(X_train), axis=-1)
print(classification_report(pred, exp))

140/140 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1746
           1       0.93      0.97      0.95       682
           2       0.90      0.98      0.94       457
           3       0.96      0.97      0.96       511
           4       0.99      0.94      0.96      1058

    accuracy                           0.97      4454
   macro avg       0.96      0.97      0.96      4454
weighted avg       0.97      0.97      0.97      4454



# Testing phase

### Load Data

In [108]:
# load test.csv file as dataframe
test = pd.read_csv('test.csv')


### Data Preprocessing

In [118]:
# Preprocessing is the same as training
data_test = pd.DataFrame(test)
data_test = data_test.drop(['created_time'], axis=1)
df = pd.DataFrame(test)
one_hot_loc = pd.get_dummies(df.loc[:,'sniffer_loc'])
df = df.drop(['sniffer_loc'], axis=1)
df = df.join(one_hot_loc)
mac = df.groupby('mac_hash', as_index=False).sum()
mac.head()


C:\Users\user\AppData\Local\Temp\ipykernel_28648\2805064473.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mac = df.groupby('mac_hash', as_index=False).sum()


,mac_hash,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,004c76bfe9f7b20f3651e0bcbd8eee9e,1,1,1,1,1,1,0,1,1,0,0,0,0,0
1,0084303d531091816387956ea26b86eb,0,1,1,1,1,0,1,1,1,1,0,0,0,0
2,00872a1ee635c2619454350b96b94103,0,1,1,1,1,1,0,1,1,1,1,1,1,1
3,00a5f69fc7719ba29c7b202f292dbafb,0,0,1,1,1,1,1,1,1,1,1,1,1,1
4,00ad41313f1f637e079c1a9d9a857d12,0,1,1,0,1,1,0,1,1,0,1,1,1,0


### Predict the result using your model

In [125]:
# X_test: feature
X_test = mac[mac.columns[1:15]]
# X_test = X.drop('mac_hash')

In [126]:
# Prediction
test = np.argmax(model.predict(X_test), axis=-1)
test

44/44 [==============================] - 0s 1ms/step


array([1, 4, 4, ..., 0, 1, 4], dtype=int64)

In [138]:
# Generate submit.csv
submit = mac[mac.columns[0]]
test_label = pd.DataFrame(test)
submit = pd.concat([submit, test_label], axis=1)
submit.to_csv('submit.csv', index = False)
submit


,mac_hash,0
0,004c76bfe9f7b20f3651e0bcbd8eee9e,1
1,0084303d531091816387956ea26b86eb,4
2,00872a1ee635c2619454350b96b94103,4
3,00a5f69fc7719ba29c7b202f292dbafb,4
4,00ad41313f1f637e079c1a9d9a857d12,4
...,...,...
1386,ff6883d04de66c6d4859928ec9d3f642,0
1387,ff7b41d3b33fb7d0b6704f1093173c40,4
1388,ffa5692a2dac937f456280c433961ea6,0
1389,ffcaf0f29c6bede7b5cd9bc807c31715,1
